# Running a Federated Cycle with Synergos

This tutorial aims to give you an understanding of how to use the synergos package to run a full federated learning cycle. 

In a federated learning system, there are many contributory participants, known as Worker nodes, which receive a global model to train on, with their own local dataset. The dataset does not leave the individual Worker nodes at any point, and remains private to the node.

The job to synchronize, orchestrate and initiate an federated learning cycle, falls on a Trusted Third Party (TTP). The TTP pushes out the global model architecture and parameters for the individual nodes to train on, calling upon the required data, based on tags, e.g "training", which points to relevant data on the individual nodes. At no point does the TTP receive, copy or access the Worker nodes' local datasets.

In this tutorial, you will go through the steps required by each participant (TTP and Worker), by simulating each of them locally with docker containers. Specifically, we will simulate a TTP and 2 Workers. 

At the end of this, we will have:
- Connected the participants
- Trained the model
- Evaluate the model

## About the Dataset and Task

The dataset used in this notebook is on dogs images, comprising 2 classes, and all images are 28 x 28 pixels. The dataset is available in the same directory as this notebook. Within the dataset directory, `data1` is for Worker 1 and `data2` is for Worker 2. The task to be carried out will be a binary classification.

The dataset we have provided is a processed subset of the [original Stanford Dogs dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/).

**Reference:**
- *DAditya Khosla, Nityananda Jayadevaprakash, Bangpeng Yao and Li Fei-Fei. Novel dataset for Fine-Grained Image Categorization. First Workshop on Fine-Grained Visual Categorization (FGVC), IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2011.*
- *J. Deng, W. Dong, R. Socher, L.-J. Li, K. Li and L. Fei-Fei, ImageNet: A Large-Scale Hierarchical Image Database. IEEE Computer Vision and Pattern Recognition (CVPR), 2009.*

## Initiating the docker containers

Before we begin, we have to start the docker containers.

Firstly, pull the required docker images with the following commands:
1. Synergos TTP (Basic):

`docker pull registry.aisingapore.net/fedlearn/synergos_ttp:syn0.7`

2. Synergos Worker:

`docker pull registry.aisingapore.net/fedlearn/synergos_worker:syn0.5`

Next, in <u>separate</u> CLI terminals, run the following command:

**Note: For Windows users, it is advisable to use powershell or command prompt based interfaces**

**Worker 1**

```docker run -v <directory dogs_binary_class/data1>:/worker/data -v <directory dogs_binary_class/outputs_1>:/worker/outputs --name worker_1 synergos_worker:v0.1.0 --id worker_1 --logging_variant basic```

**Worker 2**

```docker run -v <directory dogs_binary_class/data2>:/worker/data -v <directory dogs_binary_class/outputs_2>:/worker/outputs --name worker_2 synergos_worker:v0.1.0 --id worker_2 --logging_variant basic```

**TTP**

```docker run -p 0.0.0.0:5000:5000 -p 5678:5678 -p 8020:8020 -p 8080:8080 -v <directory dogs_binary_class/mlflow_test>:/ttp/mlflow -v <directory dogs_binary_class/ttp_data>:/ttp/data --name ttp --link worker_1 --link worker_2 synergos_ttp:v0.1.0 --id ttp --logging_variant basic -c```


Once ready, for each terminal, you should see that a Flask app is running on http://0.0.0.0:5000 of the container.

You are now ready for the next step.

## Configuration

In a new terminal, run `docker inspect bridge` and find the IPv4Address for each container. Ideally, the containers should have the following addresses:
- worker_1 address: 172.17.0.2
- worker_2 address: 172.17.0.3
- ttp address: 172.17.0.4

If not, just note the relevant IP addresses for each docker container.

Run the following cells below.

**Note: For Windows users, `host` should be Docker Desktop VM's IP. Follow [this](https://stackoverflow.com/questions/58073936/how-to-get-ip-address-of-docker-desktop-vm) on instructions to find IP**

In [1]:
from synergos import Driver

host = "localhost" # Different for Windows users
port = 5000

# Initiate Driver
driver = Driver(host=host, port=port)

## Phase 1: Connect

Submitting TTP & Participant metadata

#### 1A. TTP creates a collaboration

In [ ]:
collab_task = driver.collaborations
collab_task.create('test_collaboration')

#### 1B. TTP controller creates a project

In [ ]:
driver.projects.create(
    collab_id="test_collaboration",
    project_id="test_project",
    action="classify",
    incentives={
        'tier_1': [],
        'tier_2': [],
    }
)

#### 1C. TTP controller creates an experiment

In [ ]:
driver.experiments.create(
    collab_id="test_collaboration",
    project_id="test_project",
    expt_id="test_experiment",
    model=[
        {
            "activation": "relu",
            "is_input": True,
            "l_type": "Conv2d",
            "structure": {
                "bias": True,
                "in_channels": 1, 
                "out_channels": 3,
                "kernel_size": 5
            }
        },
        {
            "activation": "",
            "is_input": False,
            "l_type": "MaxPool2d",
            "structure": {
                "kernel_size": 3
            }
        },
        {
            "activation": "relu",
            "is_input": False,
            "l_type": "Conv2d",
            "structure": {
                "bias": True,
                "in_channels": 3, 
                "out_channels": 1,
                "kernel_size": 3
            }
        },
        {
            "activation": "",
            "is_input": False,
            "l_type": "MaxPool2d",
            "structure": {
                "kernel_size": 3
            }
        },
        {
            "activation": "relu",
            "is_input": False,
            "l_type": "Flatten",
            "structure": {}
        },
        {
            "activation": "sigmoid",
            "is_input": False,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 4, 
                "out_features": 1 
            }
        }
    ]
)

#### 1D. TTP controller creates a run

In [ ]:
driver.runs.create(
    collab_id="test_collaboration",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run",
    rounds=2, 
    epochs=1,
    base_lr=0.0005,
    max_lr=0.005,
    criterion="BCELoss"
)

#### 1E. Participants registers their servers' configurations and roles

In [ ]:
participant_resp_1 = driver.participants.create(
    participant_id="worker_1",
)

display(participant_resp_1)

participant_resp_2 = driver.participants.create(
    participant_id="worker_2",
)

display(participant_resp_2)

In [ ]:
registration_task = driver.registrations

# Add and register worker_1 node
registration_task.add_node(
    host='172.17.0.2',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)
registration_task.list_nodes()

registration_task.create(
    collab_id="test_collaboration",
    project_id="test_project",
    participant_id="worker_1",
    role="host"
)

In [ ]:
registration_task = driver.registrations
registration_task.add_node(
    host='172.17.0.3',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)
registration_task.list_nodes()

registration_task.create(
    collab_id="test_collaboration",
    project_id="test_project",
    participant_id="worker_2",
    role="guest"
)

#### 1F. Participants registers their tags for a specific project

In [ ]:
driver.tags.create(
    collab_id="test_collaboration",
    project_id="test_project",
    participant_id="worker_1",
    train=[["train"]],
    evaluate=[["evaluate"]],
    predict=[["predict"]]
)

driver.tags.create(
    collab_id="test_collaboration",
    project_id="test_project",
    participant_id="worker_2",
    train=[["train"]],
    evaluate=[["evaluate"]],
    predict=[["predict"]]
)

## Phase 2:
Alignment, Training & Optimisation

#### 2A. Perform multiple feature alignment to dynamically configure datasets and models for cross-grid compatibility

In [ ]:
driver.alignments.create(collab_id='test_collaboration',
                         project_id="test_project",
                         verbose=False,
                         log_msg=False)

#### 2B. Trigger training across the federated grid

In [ ]:
model_resp = driver.models.create(
    collab_id="test_collaboration",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run",
    log_msg=False,
    verbose=False
)
display(model_resp)

## Phase 3: EVALUATE 
Validation & Predictions

#### 3A. Perform validation(s) of combination(s)

In [ ]:
driver.validations.create(
    collab_id='test_collaboration',
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run",
    log_msg=False,
    verbose=False
)

#### 3B. Perform prediction(s) of combination(s)

In [ ]:
driver.predictions.create(
    collab_id="test_collaboration",
    tags={"test_project": [["predict"]]},
    participant_id="worker_1",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)